In [ ]:
# ! mkdir -p dataset
# ! unzip -oq work/PreTrainData.zip -d dataset
# ! pip install --upgrade paddleseg

In [ ]:
import os
import os.path as osp
import random

def create_list(gt_folder="dataset/Annotations"):
    random.seed(24)
    names = os.listdir(gt_folder)
    random.shuffle(names)
    print("Data volume: ", len(names))
    with open("dataset/train.txt", "w") as tf:
        with open("dataset/val.txt", "w") as vf:
            for idx, name in enumerate(names):
                name = name.split(".")[0]
                if idx < 17:
                    vf.write("JPEGImages/" + name + ".jpg Annotations/" + name + ".png\n")
                else:
                    tf.write("JPEGImages/" + name + ".jpg Annotations/" + name + ".png\n")
    print("Finished")

# create_list()

In [ ]:
import paddleseg.transforms as T
from paddleseg.datasets import Dataset

base_lr = 3e-4
train_lens = 200
epochs = 1000
batch_size = 16
iters = epochs * train_lens // batch_size

# 构建训练集
train_transforms = [
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomAffine(),
    T.RandomRotation(),
    T.RandomDistort(),
    T.RandomBlur(),
    T.Resize(target_size=(480, 320)),
    T.Normalize()  # 归一化
]
train_dataset = Dataset(
    transforms=train_transforms,
    dataset_root="dataset",
    num_classes=2,
    mode="train",
    train_path="dataset/train.txt",
    separator=" "
)

# 构建验证集
val_transforms = [
    T.Resize(target_size=(480, 320)),
    T.Normalize()
]
val_dataset = Dataset(
    transforms=val_transforms,
    dataset_root="dataset",
    num_classes=2,
    mode="val",
    val_path="dataset/val.txt",
    separator=" "
)

In [ ]:
import numpy as np

for data in train_dataset:
    print(data["img"].shape, data["label"].shape)
    print(np.unique(data["label"]))
    break

In [ ]:
import paddle
from paddleseg.models import OCRNet, HRNet_W48
from paddleseg.models.losses import MixedLoss, CrossEntropyLoss, LovaszSoftmaxLoss, DiceLoss

# 网络
model = OCRNet(
    num_classes=2,
    backbone=HRNet_W48(),
    backbone_indices=[0],
    pretrained="https://bj.bcebos.com/paddleseg/dygraph/cityscapes/ocrnet_hrnetw48_cityscapes_1024x512_160k/model.pdparams",
)

# 损失函数
losses = {}
losses["types"] = [MixedLoss([CrossEntropyLoss(), LovaszSoftmaxLoss(), DiceLoss()], [0.8, 0.2, 1])] * 2
losses["coef"] = [1, 0.4]

# 学习率及优化器
lr = paddle.optimizer.lr.PolynomialDecay(base_lr, decay_steps=iters, end_lr=base_lr / 5)
optimizer = paddle.optimizer.AdamW(lr, beta1=0.9, beta2=0.999, weight_decay=0.01, parameters=model.parameters())

In [ ]:
from paddleseg.core import train

train(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    optimizer=optimizer,
    save_dir="output",
    iters=iters,
    batch_size=batch_size,
    save_interval=int(iters / 50),
    log_iters=10,
    num_workers=0,
    losses=losses,
    use_vdl=True
)